In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

tf.reset_default_graph()

# 먼저 학습시킬데이터와 학습목표치 데이터를 위한 리스트를 만들어보았습니다.
train_input = []
train_label = []

# a,e,i,o,u 15만개씩 있는데이터를 하나의 txt파일로 합쳐서 불러왔습니다.
wav_all = np.loadtxt('75Raw_total.txt',np.float32)

# 불러온 데이터를 reshape하여 20000개씩 잘라내서 데이터를 구분하였습니다.
data_all = np.reshape(wav_all, [-1,20000])
x_data = data_all

# 여러 학습 형태를 돌려본 결과 1/5정도의 데이터 앞부분만으로도 어느정도 좋은 데이터를 불러올 수 있다고 생각하였습니다
x_data = x_data[:,0:4096]

# 학습목표치 데이터는 onehot방식 사용을 하기 위해서 0~4까지의 임의 목표치를 설정하였습니다.
# a = 0, e = 1, i = 2, o = 3, u = 4 로 정했습니다.
# while문을 사용하여 75개씩의 0,1,2,3,4 값을 가진 array를 만들었습니다.
target_value = []

while len(target_value) != 375:
    number = 0
    if len(target_value) == 75:
        number = 1
    elif len(target_value) == 150:
        number = 2
    elif len(target_value) == 225:
        number = 3
    elif len(target_value) == 300:
        number = 4
    else:
        number = 0
    for i in range(75):
        target_value.append(number)
    
# 위에서처럼 75개씩의 0,1,2,3,4의 값을 가진 리스트를 다시 reshape하여 onehot방식으로 변환하였습니다.
target_value = np.reshape(target_value,[-1,1])

y_data = target_value

enc = OneHotEncoder()
enc.fit(y_data)
y_data_onehot = enc.transform(y_data).toarray()

# 위에서처럼 만든 데이터들을 train_input, train_label로 설정하여 학습을 진행하였습니다.
train_input = x_data
train_label = y_data_onehot

C:\Users\Seo\.conda\envs\cuda\lib\site-packages\sklearn\preprocessing\_encoders.py:414: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [3]:

# 학습을 위한 신경망구조를 짜보았습니다.

# 위에서 train_input data의 한 데이터가 4096(64*64)크기를 가지므로 아래처럼 placeholder의 크기를 지정하였습니다.

# train_label은 5개의 목표치를 위한 onehot 방식의 array이기 때문에 참고하여 Y의 placeholder의 크기를 지정하였습니다.

# 신경망은 leaky_relu방식을 사용하였으며 dropout(학습시 신경망의 일부분을 끄고 학습시키는것)을 사용하여 학습 능률을 높였습니다.

# 풀링을 할때 2*2 필터로 풀링을 하면(padding=same으로하면 크기는 유지) 원래의 크기에서 반으로 주는것을 참고하였습니다.

# 마지막 구분하는 신경망 구조를 만들때는 일반적으로 sigmoid(0~1)를 쓰지만 여기서는 목표치가 0,1,2,3,4 이기때문에 matmul을 사용했습니다.

X = tf.placeholder(tf.float32, [None,4096])
X_wav = tf.reshape(X, [-1,64,64,1])
Y = tf.placeholder(tf.float32,[None,5])
rate = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
#    Conv     -> (?, 64, 64, 32)
#    Pool     -> (?, 32, 32, 32)
L1 = tf.nn.conv2d(X_wav, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.leaky_relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
L1 = tf.nn.dropout(L1, rate=rate)


W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
#    Conv      ->(?, 32, 32, 64)
#    Pool      ->(?, 16, 16, 64)
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.leaky_relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
L2 = tf.nn.dropout(L2, rate=rate)



W3 = tf.Variable(tf.random_normal([3, 3, 64, 128], stddev=0.01))
#    Conv      ->(?, 16, 16, 64)
#    Pool      ->(?, 8, 8, 64)
L3 = tf.nn.conv2d(L2, W3, strides=[1, 1, 1, 1], padding='SAME')
L3 = tf.nn.leaky_relu(L3)
L3 = tf.nn.max_pool(L3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
L3 = tf.nn.dropout(L3, rate=rate)



W4 = tf.Variable(tf.random_normal([3, 3, 128, 64], stddev=0.01))
#    Conv      ->(?, 8, 8, 64)
#    Pool      ->(?, 4, 4, 64)
L4 = tf.nn.conv2d(L3, W4, strides=[1, 1, 1, 1], padding='SAME')
L4 = tf.nn.leaky_relu(L4)
L4 = tf.nn.max_pool(L4, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
L4 = tf.nn.dropout(L4, rate=rate)

L4 = tf.reshape(L4, [-1, 64 * 4 * 4])


W5 = tf.get_variable("W5", shape=[64 * 4 * 4, 64], initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([64]))
L5 = tf.nn.sigmoid(tf.matmul(L4, W5) + b5)
L5 = tf.nn.dropout(L5, rate=rate)


W6 = tf.get_variable("W6", shape=[64, 5], initializer=tf.contrib.layers.xavier_initializer())
b6 = tf.Variable(tf.random_normal([5]))

logits = tf.matmul(L5, W6) + b6

Instructions for updating:
Colocations handled automatically by placer.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [4]:
# 학습을 해나가는 optimizer로 adam을 적용시켰습니다.

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)


# 변수 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())

#batch_size는 125로 하였고 2000번의 epochs로 진행했습니다.
batch_size = 125
training_epochs = 2000

print("학습을 시작합니다.")

for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(375 // batch_size)
 

        
    for i in range(total_batch):
        start = ((i+1) * batch_size) - batch_size
        end = ((i+1) * batch_size)
        batch_xs = train_input[start:end]
        batch_ys = train_label[start:end]
        feed_dict = {X: batch_xs, Y: batch_ys, rate : 0.3}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch
        
    if epoch % 500 == 0 or epoch == training_epochs-1:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

 
print('학습이 종료되었습니다.')

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

학습을 시작합니다.
Epoch: 0001 cost = 2.271681666
Epoch: 0501 cost = 0.368571584
Epoch: 1001 cost = 0.324208735
Epoch: 1501 cost = 0.282435761
Epoch: 2000 cost = 0.281722807
학습이 종료되었습니다.


'\n학습을 시작합니다.\nEpoch: 0001 cost = 1.972215215\nEpoch: 0501 cost = 0.322055269\nEpoch: 1001 cost = 0.287063878\nEpoch: 1501 cost = 0.280937931\nEpoch: 2000 cost = 0.285464690\n학습이 종료되었습니다.\n'

In [7]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print("a,e,i - Accuracy:", sess.run(accuracy, feed_dict={X:train_input[0:225], Y:train_label[0:225],rate:0}))

a,e,i - Accuracy: 1.0


In [8]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print("o,u - Accuracy:", sess.run(accuracy, feed_dict={X:train_input[225:375], Y:train_label[225:375],rate:0}))

o,u - Accuracy: 0.5


In [10]:
import random
for j in range(0,10):
    i = random.randint(225, 375)
    print("\nTarget:     ", sess.run(tf.argmax(train_label[i:i + 1], 1)))
    print("Prediction: ", sess.run(
        tf.argmax(logits, 1), feed_dict={X: train_input[i:i + 1], rate : 0}))


Target:      [4]
Prediction:  [4]

Target:      [4]
Prediction:  [4]

Target:      [4]
Prediction:  [4]

Target:      [4]
Prediction:  [4]

Target:      [3]
Prediction:  [3]

Target:      [4]
Prediction:  [3]

Target:      [4]
Prediction:  [4]

Target:      [4]
Prediction:  [4]

Target:      [3]
Prediction:  [4]

Target:      [3]
Prediction:  [4]
